In [44]:
#importing the libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import ml_metrics as metrics

In [2]:
#importing the data of the cleanned genre and plot
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [3]:
#importing the data of most frequent words/common words in the plot column for each genre
data=pd.read_csv("most_frequent_words_30.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
835,anime,help,95
836,anime,begin,94
837,anime,father,93
838,anime,attack,91


In [4]:
#making a dict for the 
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
#dict_words

In [5]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [6]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]


Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)


Wall time: 1min 11s


<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
%%time

X_train=Train['Plot_trimmed']
y_train=Train['Genre_updated']


X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.3,random_state=123,stratify=y_train)


X_test=Test['Plot_cleanned']
y_test=Test['Genre_updated']

Wall time: 323 ms


In [8]:
%%time
tfidf_vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,2))

X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 

X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) 


X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

Wall time: 2.4 s


In [9]:
cosine_sim= cosine_similarity(X_train_vectors_tfidf , X_train_vectors_tfidf )

In [10]:
print(cosine_sim)

[[1.         0.1365039  0.05496735 ... 0.06114691 0.05109519 0.        ]
 [0.1365039  1.         0.03812398 ... 0.1286287  0.1747382  0.01574389]
 [0.05496735 0.03812398 1.         ... 0.10606514 0.08400621 0.1060241 ]
 ...
 [0.06114691 0.1286287  0.10606514 ... 1.         0.07129796 0.07263758]
 [0.05109519 0.1747382  0.08400621 ... 0.07129796 1.         0.03934499]
 [0.         0.01574389 0.1060241  ... 0.07263758 0.03934499 1.        ]]


In [11]:
indices = pd.Series(Train.index, index=Train['Title']).drop_duplicates()
indices

Title
The Great Train Robbery                                  6
The Suburbanite                                          7
Dream of a Rarebit Fiend                                10
From Leadville to Aspen: A Hold-Up in the Rockies       11
From Leadville to Aspen: A Hold-Up in the Rockies       12
                                                     ...  
Particle (film)                                      42526
Çalgı Çengi İkimiz                                   42531
Olanlar Oldu                                         42532
Non-Transferable                                     42533
İstanbul Kırmızısı                                   42534
Length: 35692, dtype: int64

In [47]:
def get_recommendations(title, cosine_sim, indices):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:20]
    movie_indices = [i[0] for i in sim_scores]
    return set(Train['Title'].iloc[movie_indices])

In [48]:
recom=list(get_recommendations('The Suburbanite', cosine_sim, indices))
recom

['The Last Blitzkrieg',
 'Daleks – Invasion Earth: 2150 A.D.',
 'The First Great Train Robbery',
 'Horsemen',
 "Mother's Meat and Freud's Flesh",
 'The Eve of St. Mark',
 'Edge of Darkness',
 'Atlas Shrugged: Part I',
 'Ivy',
 'Perfect Stranger',
 'Cheap Thrills (film)',
 'The Dark Tower',
 'Violet & Daisy',
 'The Unfaithful',
 'Young Billy Young',
 'The Wizard of Gore',
 "The Wildcats of St Trinian's",
 'Good-bye, My Lady',
 'Blind Dating']

In [49]:
metrics.average_precision.apk(list(indices.keys()),recom,10)

1.0

In [50]:
metrics.average_precision.mapk(list(indices.keys()),recom,10)

0.34234753550543023